In [2]:
# อนุญาตเข้าถึง google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import re
import pandas as pd

In [5]:
### กำหนด data type ที่เหมาะสมกับ attribute values (Custom data types) ###


def guess_column_types(file_path, delimiter=',', has_headers=True):
    try:
        # Read the CSV file using the specified delimiter and header settings
        df = pd.read_csv(file_path, sep=delimiter,low_memory=False, header=0 if has_headers else None)

        # Initialize a dictionary to store column data types
        column_types = {}

        # Loop through columns and infer data types
        for column in df.columns:
            # sample_values = df[column].dropna().sample(min(5, len(df[column])), random_state=42)

            # Check for datetime format "YYYY-MM-DD HH:MM:SS"
            is_datetime = all(re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(value)) for value in df[column])

            # Check for date format "YYYY-MM-DD"
            is_date = all(re.match(r'\d{4}-\d{2}-\d{2}', str(value)) for value in df[column])

            # Assign data type based on format detection
            if is_datetime:
                inferred_type = 'datetime64'
            elif is_date:
                inferred_type = 'date'
            else:
                inferred_type = pd.api.types.infer_dtype(df[column], skipna=True)

            column_types[column] = inferred_type

        return (True, column_types)  # Return success and column types
    except pd.errors.ParserError:
        return (False, str(e))  # Return error message

In [6]:
file_path = '/gdrive/MyDrive/Colab Notebooks/Data Analytics using Python and Power Bi/File/LoanStats_web.csv'
result, column_types_or_error = guess_column_types(file_path)

if result:
    print("Column Types:", column_types_or_error)
else:
    print("Error:", column_types_or_error)

Column Types: {'id': 'string', 'member_id': 'floating', 'loan_amnt': 'floating', 'funded_amnt': 'floating', 'funded_amnt_inv': 'floating', 'term': 'string', 'int_rate': 'string', 'installment': 'floating', 'grade': 'string', 'sub_grade': 'string', 'emp_title': 'string', 'emp_length': 'string', 'home_ownership': 'string', 'annual_inc': 'floating', 'verification_status': 'string', 'issue_d': 'string', 'loan_status': 'string', 'pymnt_plan': 'string', 'url': 'floating', 'desc': 'string', 'purpose': 'string', 'title': 'string', 'zip_code': 'string', 'addr_state': 'string', 'dti': 'floating', 'delinq_2yrs': 'floating', 'earliest_cr_line': 'string', 'inq_last_6mths': 'floating', 'mths_since_last_delinq': 'floating', 'mths_since_last_record': 'floating', 'open_acc': 'floating', 'pub_rec': 'floating', 'revol_bal': 'floating', 'revol_util': 'string', 'total_acc': 'floating', 'initial_list_status': 'string', 'out_prncp': 'floating', 'out_prncp_inv': 'floating', 'total_pymnt': 'floating', 'total_p

In [7]:
# เปลี่ยน data type บางตัว ให้เหมาะสมกับ python's environment และ MSSQL
column_types_corrected = {col: ('datetime64' if t == 'date' else 'float64' if t == 'floating' else t) \
                          for col, t in column_types_or_error.items()}

In [9]:
###** นำ dictionary ที่บรรจุ ชื่อ col และ data type มาใช้เป็น parameter ของการทำ pd.read_csv ( ) อ่าน csv มาเป็น dataframe ###
# workshop 3 ให้นำ model ของตัวเองมาใส่โดยนำ column ที่เขียนบนกระดานมาใช้
raw_df = pd.read_csv(file_path, dtype=column_types_corrected)\
[['application_type', 'annual_inc', 'annual_inc_joint', 'dti', 'dti_joint', 'emp_length'\
 ,'issue_d','int_rate','emp_title']]

In [10]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432466 entries, 0 to 1432465
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1323970 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   emp_title         1302202 non-null  string 
dtypes: float64(4), string(5)
memory usage: 98.4 MB


In [11]:
##**
# ตั้งค่าการแสดงผลของ pandas ให้ไม่ใช้ scientific notation และแสดงตัวเลขเต็ม
pd.set_option('display.float_format', '{:.2f}'.format)

In [12]:
##**
raw_df.describe(include='all')

,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,emp_title
count,1432440,1432440.00,112083.00,1430856.00,112081.00,1323970,1432440,1432440,1302202
unique,2,NaN,NaN,NaN,NaN,11,30,258,245698
top,Individual,NaN,NaN,NaN,NaN,10+ years,Mar-2017,11.49%,Teacher
freq,1320357,NaN,NaN,NaN,NaN,478304,148724,49032,26216
mean,NaN,81034.58,126843.25,19.21,19.09,NaN,NaN,NaN,NaN
std,NaN,134183.36,77982.26,15.97,7.84,NaN,NaN,NaN,NaN
min,NaN,0.00,5693.51,-1.00,0.00,NaN,NaN,NaN,NaN
25%,NaN,48000.00,85000.00,11.99,13.35,NaN,NaN,NaN,NaN
50%,NaN,68000.00,113000.00,18.00,18.64,NaN,NaN,NaN,NaN
75%,NaN,96667.00,150000.00,24.83,24.42,NaN,NaN,NaN,NaN


In [13]:
#** แทนที่ค่า null ในคอลัมน์ 'emp_length' ด้วย 'N/A'
raw_df['emp_length'] = raw_df['emp_length'].fillna('N/A')

In [14]:
##**
raw_df['emp_length'].value_counts()

emp_length
10+ years    478304
2 years      130251
< 1 year     118393
3 years      116802
N/A          108496
1 year        95926
5 years       88686
4 years       86767
6 years       62261
8 years       51222
7 years       48965
9 years       46393
Name: count, dtype: Int64

In [15]:
##**
raw_df[raw_df['dti'].isnull()]

,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,emp_title
449,Joint App,0.00,40000.00,NaN,13.65,N/A,Mar-2016,9.16%,<NA>
2224,Joint App,0.00,115000.00,NaN,20.20,N/A,Mar-2016,11.99%,<NA>
5419,Joint App,0.00,56000.00,NaN,30.30,N/A,Mar-2016,9.16%,<NA>
10851,Joint App,0.00,65000.00,NaN,8.71,N/A,Mar-2016,12.99%,<NA>
16023,Joint App,0.00,50000.00,NaN,9.84,N/A,Mar-2016,22.45%,<NA>
...,...,...,...,...,...,...,...,...,...
1430415,Joint App,0.00,72000.00,NaN,17.81,N/A,Jan-2019,19.92%,<NA>
1431977,Joint App,0.00,166442.00,NaN,9.21,N/A,Jan-2019,8.19%,<NA>
1432116,Joint App,0.00,103000.00,NaN,18.89,5 years,Jan-2019,7.56%,Clinician
1432464,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>


In [16]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432466 entries, 0 to 1432465
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1432466 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   emp_title         1302202 non-null  string 
dtypes: float64(4), string(5)
memory usage: 98.4 MB


In [17]:
# ตัวอย่างโค้ดเพื่อตรวจสอบว่า ทุกครั้งที่ dti เป็น null แล้ว annual_inc_joint จะมากกว่า 0 เสมอ
condition = (raw_df['dti'].isnull()) & (raw_df['annual_inc_joint'] >= 0)
# คำนวณจำนวนแถวที่ตรงตามเงื่อนไข
matching_rows = raw_df[condition]

# ตรวจสอบว่ามีแถวใดที่ตรงตามเงื่อนไขหรือไม่
if not matching_rows.empty:
    print(f"There are {len(matching_rows)} rows where 'dti' is NULL and 'annual_inc_joint' >= 0.")
else:
    print("No rows match the condition where 'dti' is NULL and 'annual_inc_joint' >= 0.")

There are 1584 rows where 'dti' is NULL and 'annual_inc_joint' >= 0.


In [18]:
# ตัวอย่างโค้ดเพื่อตรวจสอบว่า ทุกครั้งที่ dti เป็น null แล้ว annual_inc_joint จะน้อยกว่าหรือเท่ากับ 0 เสมอ
condition = (raw_df['dti'].isnull()) & (raw_df['annual_inc_joint'] <= 0)
# คำนวณจำนวนแถวที่ตรงตามเงื่อนไข
matching_rows = raw_df[condition]

# ตรวจสอบว่ามีแถวใดที่ตรงตามเงื่อนไขหรือไม่
if not matching_rows.empty:
    print(f"There are {len(matching_rows)} rows where 'dti' is NULL and 'annual_inc_joint' <= 0.")
else:
    print("No rows match the condition where 'dti' is NULL and 'annual_inc_joint' <= 0.")

No rows match the condition where 'dti' is NULL and 'annual_inc_joint' <= 0.


In [ ]:
## ไม่น่าจะมีอะไรผิดข้อเท็จจริง เกี่ยวกับ dti ดังนั้น เราจะลองปล่อย dti เป็น null ไปก่อน

In [19]:
# ตัวอย่างโค้ดเพื่อตรวจสอบว่่า ทุกครั้งที่ dti_joint เป็น null แล้ว annual_inc_joint เป็น null ด้วย
condition = (raw_df['dti_joint'].isnull()) & (raw_df['annual_inc_joint'].isnull())
# คำนวณจำนวนแถวที่ตรงตามเงื่อนไข
matching_rows = raw_df[condition]

# แสดงจำนวนแถวที่ตรงตามเงื่อนไข
print(f"Number of rows where both 'dti_joint' and 'annual_inc_joint' are NULL: {len(matching_rows)}")

Number of rows where both 'dti_joint' and 'annual_inc_joint' are NULL: 1320383


In [20]:
matching_rows

,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,emp_title
0,Individual,53000.00,NaN,20.99,NaN,6 years,Mar-2016,11.99%,LOGISTICS COORDINATOR
1,Individual,52000.00,NaN,18.86,NaN,9 years,Mar-2016,15.31%,Senior Case Assistant
2,Individual,73000.00,NaN,10.85,NaN,5 years,Mar-2016,10.75%,Vp
3,Individual,138000.00,NaN,12.44,NaN,10+ years,Mar-2016,12.99%,Administration
4,Individual,134000.00,NaN,25.69,NaN,10+ years,Mar-2016,18.25%,Electrician
...,...,...,...,...,...,...,...,...,...
1432461,Individual,32000.00,NaN,20.89,NaN,10+ years,Jan-2019,16.14%,Estimator/Supervisor
1432462,Individual,72000.00,NaN,7.02,NaN,5 years,Jan-2019,11.31%,MATERIAL HANDLER
1432463,Individual,65000.00,NaN,29.52,NaN,7 years,Jan-2019,18.94%,sr register csa
1432464,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>


In [ ]:
##จากข้างบนทำให้สงสัยว่า ทุกครั้งที่ application_type = <NA> แล้ว emp_length = N/A และ col. ที่เหลืออื่นๆ เป็น null

In [21]:
# ตัวอย่างโค้ดในการตรวจสอบเงื่อนไข
condition = (raw_df['application_type'].isnull()) & (raw_df['emp_length'] == 'N/A')

# ลิสต์ของคอลัมน์ที่ต้องการตรวจสอบว่าเป็น null ยกเว้น 'application_type' และ 'emp_length'
columns_to_check = ['annual_inc', 'annual_inc_joint', 'dti', 'dti_joint']

# ตรวจสอบในแถวที่ตรงตามเงื่อนไขของ 'application_type' และ 'emp_length'
filtered_rows = raw_df.loc[condition]

# นับจำนวนแถวที่คอลัมน์อื่นๆ ที่เหลือเป็น null
count_matching_rows = 0
for index, row in filtered_rows.iterrows():
    if row[columns_to_check].isnull().all():  # ตรวจสอบว่าทุกคอลัมน์ใน columns_to_check เป็น null
        count_matching_rows += 1

print(f"Number of rows where 'application_type' is <NA>, 'emp_length' is N/A, and other specified columns are NULL: {count_matching_rows}")

filtered_rows

Number of rows where 'application_type' is <NA>, 'emp_length' is N/A, and other specified columns are NULL: 26


,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,emp_title
133887,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
133888,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
231743,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
231744,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
330865,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
330866,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
434413,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
434414,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
531194,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>
531195,<NA>,NaN,NaN,NaN,NaN,N/A,<NA>,<NA>,<NA>


In [42]:
filtered_rows.shape

(26, 9)

In [22]:
filteredRow_df = raw_df[raw_df['application_type']!='<NA>']

In [23]:
filteredRow_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1432440 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   emp_title         1302202 non-null  string 
dtypes: float64(4), string(5)
memory usage: 109.3 MB


In [24]:
filteredRow_df.groupby('application_type').size()

application_type
Individual    1320357
Joint App      112083
dtype: int64

In [ ]:
##** หากจำเป็นต้องมี annual_inc และ annual_inc_joint ไว้ใน fact table
##   เนื่องจาก การมี annual_inc และ annual_inc_joint ไว้ใน dimension table นั้น
##   จะทำให้เกิด pk ใน dimension table เยอะเกินไป
##   ตรงนี้จึงเป็ฯจุดเริ่มต้นการ วิเคราะห์ และ แสดงให้เห็นว่า เหตุใด จึงควรเก็บ null ไว้ **##

In [25]:
filteredRow_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1432440 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   emp_title         1302202 non-null  string 
dtypes: float64(4), string(5)
memory usage: 109.3 MB


In [53]:
# หาค่า mean ของ dti_joint
average_dti_joint = filteredRow_df['dti_joint'].mean()
# หาค่า mean ของ annual_inc_joint
average_annual_inc_joint = filteredRow_df['annual_inc_joint'].mean()

print(f"Average dti_joint of the co-borrower: {average_dti_joint}")
print(f"Average annual joint income of the co-borrowers: {average_annual_inc_joint}")

Average dti_joint of the co-borrower: 19.08868309526146
Average annual joint income of the co-borrowers: 126843.24936573785


In [78]:
# แทนที่ด้วยค่าเฉลี่ย (mean)
average_dti_joint_a = filteredRow_df['dti_joint'].fillna(19.08).mean()
average_annual_inc_joint_print = filteredRow_df['annual_inc_joint'].fillna(126843.24).mean()

print(f"หาค่ากลางหลังจากเติมค่าเฉลี่ยของ dti คือ : {average_dti_joint_a}")
print(f"หาค่ากลางหลังจากเติมค่าเฉลี่ยของ annual_inc คือ : {average_annual_inc_joint_print}")

หาค่ากลางหลังจากเติมค่าเฉลี่ยของ dti คือ : 19.080679407165395
หาค่ากลางหลังจากเติมค่าเฉลี่ยของ annual_inc คือ : 126843.24073283342


In [55]:
# ดูค่า mean หลังจากเติม 0 แทนที่
average_dti_joint = filteredRow_df['dti_joint'].fillna(0).mean()
average_annual_inc_joint = filteredRow_df['annual_inc_joint'].fillna(0).mean()

print(f"Average dti_joint of the co-borrower: {average_dti_joint}")
print(f"Average annual joint income of the co-borrowers: {average_annual_inc_joint}")

Average dti_joint of the co-borrower: 1.4935904400882407
Average annual joint income of the co-borrowers: 9925.003433763366


In [56]:
# หาค่า mean ของ dti และ annual_inc
average_dti = filteredRow_df['dti'].mean()
average_annual_inc = filteredRow_df['annual_inc'].mean()
print(f"หาค่าเฉลี่ยนของ dti คือ : {average_dti}")
print(f"หาค่าเฉลี่ยนของ annual_inc คือ : {average_annual_inc}")

หาค่าเฉลี่ยนของ dti คือ : 19.21187229881973
หาค่าเฉลี่ยนของ annual_inc คือ : 81034.5829329675


In [ ]:
## จะเห็นได้ว่า การแทนที่ null ด้วย 0 ทำให้ค่าเฉลี่ยผิดเพี้ยนไป
## สิ้นสุด การวิเคราะห์ และ แสดงให้เห็นว่า เหตุใด จึงควรเก็บ null ไว้ ##

In [57]:
filteredRow_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   application_type  1432440 non-null  string 
 1   annual_inc        1432440 non-null  float64
 2   annual_inc_joint  112083 non-null   float64
 3   dti               1430856 non-null  float64
 4   dti_joint         112081 non-null   float64
 5   emp_length        1432440 non-null  string 
 6   issue_d           1432440 non-null  string 
 7   int_rate          1432440 non-null  string 
 8   emp_title         1302202 non-null  string 
dtypes: float64(4), string(5)
memory usage: 109.3 MB


In [58]:
## **
# ทำสำเนาตัวแปร โดยหลังจากนี้ หากเกิดความเปลี่ยนแปลงกับ ตัวแปรใหม่ (df_prepared) จะไม่ส่งผลใดๆ ต่อตัวแปรเดิม (noNull_df)
## df_prepared = noNull_df.copy()
df_prepared = filteredRow_df.copy() ##** ณ จุดนี้ filteredRow_df ยังคงมี null แฝงอยู่

# เปลี่ยน data type เป็น datetime สำหรับ col: issue_d
df_prepared['issue_d'] = pd.to_datetime(df_prepared['issue_d'], format='%b-%Y')

# นำเครื่องหมาย % ออกจากค่าใน col: int_rate แล้วเปลี่ยน data type เป็น float
if df_prepared['int_rate'].dtype == 'string':
    df_prepared['int_rate'] = df_prepared['int_rate'].str.rstrip('%').astype('float') / 100.0

In [59]:
df_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   application_type  1432440 non-null  string        
 1   annual_inc        1432440 non-null  float64       
 2   annual_inc_joint  112083 non-null   float64       
 3   dti               1430856 non-null  float64       
 4   dti_joint         112081 non-null   float64       
 5   emp_length        1432440 non-null  string        
 6   issue_d           1432440 non-null  datetime64[ns]
 7   int_rate          1432440 non-null  float64       
 8   emp_title         1302202 non-null  string        
dtypes: datetime64[ns](1), float64(5), string(3)
memory usage: 109.3 MB


In [60]:
df_prepared.groupby('issue_d').size()

issue_d
2016-01-01     32366
2016-02-01     39529
2016-03-01     61992
2016-04-01     36432
2016-05-01     28403
2016-06-01     33019
2016-07-01     34696
2016-08-01     36280
2016-09-01     28144
2016-10-01     32772
2016-11-01     34591
2016-12-01     36183
2017-01-01    127340
2017-02-01    111052
2017-03-01    148724
2018-01-01     36347
2018-02-01     32746
2018-03-01     38771
2018-04-01     42928
2018-05-01     46311
2018-06-01     41533
2018-07-01     43089
2018-08-01     46079
2018-09-01     39026
2018-10-01     46305
2018-11-01     41973
2018-12-01     40134
2019-01-01     43584
2019-02-01     35985
2019-03-01     36106
dtype: int64

In [61]:
df_prepared.groupby('int_rate').size()

int_rate
0.05     8613
0.05    36953
0.06      493
0.06     5019
0.06     2968
        ...  
0.31     1223
0.31      676
0.31      484
0.31      324
0.31      464
Length: 258, dtype: int64

In [62]:
## dimension table เป็นสิ่งที่อยู่ใน data warehouse ไม่ใช่ python
## แต่เราจะใช้ python ไป create dim. table ใน data warehouse พร้อม insert ข้อมูลลงไปด้วย
## ด้วยเหตุนี้ ครูเอ้จึงเสนอ idea สำหรับการใช้ python พัฒนา dim. table ดังนี้
## (1) สร้าง dataframe ขึ้นมาใหม่ เพื่อทำหน้าที่เป็น dim. table โดยมีเฉพาะ col. ตาม dimensional model ที่ออกแบบไว้แล้ว
## (2) สร้าง col. ใหม่ขึ้นมาบน dataframe ข้อ (1) เพื่อจะใช้เป็น primary key สำหรับ dim. table

# ทำข้อ (1) สำหรับ emp
##home_ownership_dim = df_prepared[['home_ownership']].drop_duplicates().reset_index(drop=True)
##home_ownership_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ home_ownership
#home_ownership_dim['home_ownership_id'] = home_ownership_dim.index

# ทำข้อ (1) สำหรับ application
##loan_status_dim = df_prepared[['loan_status']].drop_duplicates().reset_index(drop=True)
##loan_status_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ loan_status
#loan_status_dim['loan_status_id'] = loan_status_dim.index

# ทำข้อ (1) สำหรับ สำหรับ issue_d
issue_d_dim = df_prepared[['issue_d']].drop_duplicates().reset_index(drop=True)
issue_d_dim['month'] = issue_d_dim['issue_d'].dt.month
issue_d_dim['year'] = issue_d_dim['issue_d'].dt.year
issue_d_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ สำหรับ issue_d
issue_d_dim['issue_d_id'] = issue_d_dim.index

In [63]:
issue_d_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   index       30 non-null     int64         
 1   issue_d     30 non-null     datetime64[ns]
 2   month       30 non-null     int32         
 3   year        30 non-null     int32         
 4   issue_d_id  30 non-null     int64         
dtypes: datetime64[ns](1), int32(2), int64(2)
memory usage: 1.1 KB


In [64]:
# sort value issue_d
issue_d_dim.sort_values(by=['year','month'])

,index,issue_d,month,year,issue_d_id
2,2,2016-01-01,1,2016,2
1,1,2016-02-01,2,2016,1
0,0,2016-03-01,3,2016,0
5,5,2016-04-01,4,2016,5
4,4,2016-05-01,5,2016,4
3,3,2016-06-01,6,2016,3
8,8,2016-07-01,7,2016,8
7,7,2016-08-01,8,2016,7
6,6,2016-09-01,9,2016,6
11,11,2016-10-01,10,2016,11


In [68]:
##** เพิ่ม application_type_dim และ emp_length_dim

# สร้าง application_type_dim
application_type_dim = df_prepared[['application_type']].drop_duplicates().reset_index(drop=True)
application_type_dim.reset_index(inplace=True)
application_type_dim['application_type_id'] = application_type_dim.index

# สร้าง emp_length_dim
emp_length_dim = df_prepared[['emp_length','emp_title']].drop_duplicates().reset_index(drop=True)
emp_length_dim.reset_index(inplace=True)
emp_length_dim['emp_length_id'] = emp_length_dim.index

In [69]:
application_type_dim

,index,application_type,application_type_id
0,0,Individual,0
1,1,Joint App,1


In [71]:
emp_length_dim.sort_values(by=['emp_length'])

,index,emp_length,emp_title,emp_length_id
189522,189522,1 year,Physician billing Specialist,189522
68737,68737,1 year,Patient Care Representative,68737
68748,68748,1 year,XSP,68748
353419,353419,1 year,It Support Specialist,353419
260122,260122,1 year,Outside Sales / Technician,260122
...,...,...,...,...
48761,48761,N/A,Inventory Specialist,48761
88811,88811,N/A,Caregiver/nanny,88811
162008,162008,N/A,Medical Receptionist,162008
75634,75634,N/A,Host,75634


In [93]:
# ทำข้อ (1)
issue_d_map = issue_d_dim.set_index('issue_d')['issue_d_id'].to_dict()

In [94]:
#**
# สร้าง key mapping สำหรับ application_type_dim
application_type_map = application_type_dim.set_index('application_type')['application_type_id'].to_dict()

# สร้าง key mapping สำหรับ emp_length_dim
emp_length_map = emp_length_dim.set_index('emp_length')['emp_length_id'].to_dict()

In [95]:
# ทำข้อ (2)
loans_fact_workshop = df_prepared.copy()
loans_fact_workshop['issue_d_id'] = loans_fact_workshop['issue_d'].map(issue_d_map)

# เชื่อมค่า application_type ใน loans_fact กับ application_type_map
loans_fact_workshop['application_type_id'] = loans_fact_workshop['application_type'].map(application_type_map)

# เชื่อมค่า emp_length ใน loans_fact กับ emp_length_map
loans_fact_workshop['emp_length_id'] = loans_fact_workshop['emp_length'].map(emp_length_map)


# เลือกคอลัมน์ที่จำเป็นสำหรับ Fact Table
loans_fact_workshop = loans_fact_workshop[['application_type','dti', 'dti_joint','annual_inc','annual_inc_joint','int_rate','emp_length_id'\
                         , 'issue_d_id','application_type_id']]

In [ ]:
'''
annual_inc        1432440 non-null  float64
 2   annual_inc_joint
'''

In [96]:
loans_fact_workshop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1432440 entries, 0 to 1432463
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   application_type     1432440 non-null  string 
 1   dti                  1430856 non-null  float64
 2   dti_joint            112081 non-null   float64
 3   annual_inc           1432440 non-null  float64
 4   annual_inc_joint     112083 non-null   float64
 5   int_rate             1432440 non-null  float64
 6   emp_length_id        1432440 non-null  int64  
 7   issue_d_id           1432440 non-null  int64  
 8   application_type_id  1432440 non-null  int64  
dtypes: float64(5), int64(3), string(1)
memory usage: 109.3 MB


In [ ]:
### เริ่มต้นเปลี่ยนเป็น raw cell ชั่วคราว ###

In [98]:
## [optional] ทดสอบการ join ทุก dataframe ทั้ง fact และ dim. เข้าด้วยกัน
loan_fact_with_application = pd.merge(loans_fact_workshop, application_type_dim, on='application_type_id', how='left', suffixes=('','_application_type'))
loan_fact_with_emp = pd.merge(loans_fact_workshop, emp_length_dim, on='emp_length_id', how='left', suffixes=('','_emp_lenght'))


# Join ผลลัพธ์กับ `issue_d_dim` โดยใช้ 'issue_d_id'
final_df = pd.merge(loan_fact_with_emp,issue_d_dim, on='issue_d_id', how='left', suffixes=('', '_issue_d'))

In [99]:
##**
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องมี row เท่ากันกับ dataframe เดิม
## print(f"จำนวนแถวใน noNull_df: {noNull_df.shape[0]}")
print(f"จำนวนแถวใน filteredRow_df: {filteredRow_df.shape[0]}")
print(f"จำนวนแถวใน final_df: {final_df.shape[0]}")

จำนวนแถวใน filteredRow_df: 1432440
จำนวนแถวใน final_df: 1432440


In [101]:
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องไม่มี NULL เลย
print("จำนวนค่า Null ใน final_df หลังจากการ join:")
print(final_df.isnull().sum())

จำนวนค่า Null ใน final_df หลังจากการ join:
application_type             0
dti                       1584
dti_joint              1320359
annual_inc                   0
annual_inc_joint       1320357
int_rate                     0
emp_length_id                0
issue_d_id                   0
application_type_id          0
index                        0
emp_length                   0
emp_title                    0
index_issue_d                0
issue_d                      0
month                        0
year                         0
dtype: int64


In [ ]:
### สิ้นสุดเปลี่ยนเป็น raw cell ชั่วคราว ###

In [102]:
# External IP Address : 173.255.112.98, old ip : 35.193.72.209
server = '35.193.184.92'
database = 'TestDB'
username = 'SA'
password = 'Passw0rd123456'

In [103]:
issue_d_dim.drop(columns=['index'], inplace=True)
emp_length_dim.drop(columns=['index'], inplace=True)
application_type_dim.drop(columns=['index'], inplace=True)

In [104]:
# install packet pymssql in colab workbook
!pip install pymssql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 24.3 MB/s eta 0:00:00


In [105]:
from sqlalchemy import create_engine
import urllib

# ตั้งค่าการเชื่อมต่อกับ MSSQL โดยใช้ข้อมูลที่เหมาะสมกับสภาพแวดล้อมของคุณ
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=server;DATABASE=database;UID=username;PWD=password")
##### engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
# Using pymssql
engine = create_engine(f'mssql+pymssql://{username}:{password}@{server}/{database}')


# นำเข้าข้อมูล Dimension Tables ไปยัง MSSQL ให้ตรงกับขั้นตอนการเตรียมข้อมูลที่คุณทำไว้
emp_length_dim.to_sql('emp_workshop', con=engine, if_exists='replace', index=False)
application_type_dim.to_sql('application_type_workshop', con=engine, if_exists='replace', index=False)
issue_d_dim.to_sql('issue_d_dim_workshop2', con=engine, if_exists='replace', index=False)



# นำเข้าข้อมูล Fact Table ไปยัง MSSQL โดยใช้ข้อมูลที่เตรียมไว้
loans_fact_workshop.to_sql('loans_fact_workshop', con=engine, if_exists='replace', index=False)

189